In [18]:
# example of plotting the adam search on a contour plot of the test function
import numpy as np
from math import sqrt
from numpy import asarray
from numpy import arange
from numpy.random import rand
from numpy.random import seed
from numpy import meshgrid
from matplotlib import pyplot
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import FuncAnimation
%matplotlib widget 

In [19]:
# objective function
def objective(x, y):
    return 100 * (y - x * x) * (y - x * x) + (1 - x) * (1 - x)

In [20]:
# derivative of objective function
def derivative(x, y):
    return asarray([400 * (x**3 - x * y) + 2 * x - 2, 200 * (y - x**2)])

In [21]:
# seed the pseudo random number generator
seed(121)
# define range for input
#bounds = asarray([[-2.0, 2.0], [-1.0, 3.0]])
bounds = asarray([[0.6, 1.1], [0.6, 1.1]])
# define the total iterations
n_iter = 500
# steps size
alpha = 0.01
# factor for average gradient
beta1 = 0.8
# factor for average squared gradient
beta2 = 0.999

K_P = 0.001
K_I = 0.0006
K_D = 0.0003
a = 20
alpha = 0.95



In [22]:
def adam(x, objective, derivative, bounds, n_iter, alpha, beta1, beta2, eps=1e-8):
    solutions = list()

    score = objective(x[0], x[1])
    # initialize first and second moments
    m = [0.0 for _ in range(bounds.shape[0])]
    v = [0.0 for _ in range(bounds.shape[0])]
    # run the gradient descent updates
    for t in range(n_iter):
        # calculate gradient g(t)
        g = derivative(x[0], x[1])
        # build a solution one variable at a time
        for i in range(bounds.shape[0]):
            # m(t) = beta1 * m(t-1) + (1 - beta1) * g(t)
            m[i] = beta1 * m[i] + (1.0 - beta1) * g[i]
            # v(t) = beta2 * v(t-1) + (1 - beta2) * g(t)^2
            v[i] = beta2 * v[i] + (1.0 - beta2) * g[i] ** 2
            # mhat(t) = m(t) / (1 - beta1(t))
            mhat = m[i] / (1.0 - beta1 ** (t + 1))
            # vhat(t) = v(t) / (1 - beta2(t))
            vhat = v[i] / (1.0 - beta2 ** (t + 1))
            # x(t) = x(t-1) - alpha * mhat(t) / (sqrt(vhat(t)) + ep)
            x[i] = x[i] - alpha * mhat / (sqrt(vhat) + eps)
        # evaluate candidate point
        score = objective(x[0], x[1])
        # keep track of solutions
        solutions.append(x.copy())
        # report progress
        print(">%d f(%s) = %.5f" % (t, x, score))
    return solutions

In [23]:
def PIDam(x, objective, derivative, n_iter, K_P, K_I, K_D, a):
    solutions = list()
    score = objective(x[0], x[1])

    G = np.zeros((a, 2))

    for t in range(n_iter):
        G = np.roll(G, 1)
        G[0] = derivative(x[0], x[1])
        a1 = np.dot(G[0], (K_P + K_I + K_D))
        a2 = np.dot(G[1], K_I - K_D)
        a3 = np.dot(np.mean(G[2:], axis=0), K_I)
        x -= a1 + a2 + a3
        score = objective(x[0], x[1])
        solutions.append(x.copy())

        print(">%d f(%s) = %.5f" % (t, x, score))

    return solutions

In [24]:
def PIDam2(x, objective, derivative, n_iter, K_P, K_I, K_D, alpha):
    solutions = list()
    solutions.append(x.copy())
    score = objective(x[0], x[1])
    M = np.zeros(2)
    G = np.zeros((2, 2))

    for t in range(n_iter):
        G = np.roll(G, 1)
        G[0] = derivative(x[0], x[1])
        # G[0] = np.divide(G[0], max(np.max(np.abs(G[0])), 1))
        M = alpha * M + G[0]

        x -= K_P * G[0] + K_I * M + K_D * (G[0] - G[1])

        score = objective(x[0], x[1])
        solutions.append(x.copy())
        if score < 0.01:
            M = np.zeros(2)

        print(">%d f(%s) = %.5f" % (t, x, score))
        if score < 0.001:
            break

    return solutions

In [25]:
def SGD(x, objective, derivative, n_iter):
    eta = 0.001
    
    solutions = list()
    solutions.append(x.copy())
    score = objective(x[0], x[1])

    for t in range(n_iter):
        G = derivative(x[0], x[1])
        x -= G * eta
        score = objective(x[0], x[1])
        solutions.append(x.copy())

        print(">%d f(%s) = %.5f" % (t, x, score))

    return solutions

In [26]:
def SGDmom(x, objective, derivative, n_iter):
    rho = 0.95
    eta = 0.00005

    solutions = list()
    solutions.append(x.copy())

    score = objective(x[0], x[1])
    M = np.array([0, 0])

    for t in range(n_iter):
        G = derivative(x[0], x[1])
        M = np.add(M*rho, G*eta)

        x -= M
        score = objective(x[0], x[1])
        solutions.append(x.copy())

        print(">%d f(%s) = %.5f" % (t, x, score))

    return solutions

In [27]:
def GradPID(x, objective, derivative, n_iter):
    rho = 0.95
    eta = 0.0006

    solutions = list()
    solutions.append(x.copy())

    score = objective(x[0], x[1])
    M = np.array([0., 0.])
    Gprev = np.array([0., 0.])

    for t in range(n_iter):
        G = derivative(x[0], x[1])*eta
        M = M*rho + G
        D = (G - Gprev)

        x -= 0.1*M + G + 2*D
        score = objective(x[0], x[1])
        solutions.append(x.copy())

        Gprev = G.copy()

        print(">%d f(%s) = %.5f" % (t, x, score))

    return solutions


In [28]:
def signSGD(x, objective, derivative, n_iter):
    solutions = list()
    score = objective(x[0], x[1])

    for t in range(n_iter):
        G = derivative(x[0], x[1])
        G = np.sign(G)
        x -= np.dot(G, 0.01)
        score = objective(x[0], x[1])
        solutions.append(x.copy())

        print(">%d f(%s) = %.5f" % (t, x, score))

    return solutions

In [ ]:
def NormGD(x, objective, derivative, n_iter):
    eta = 0.001

    solutions = list()
    solutions.append(x.copy())
    score = objective(x[0], x[1])

    for t in range(n_iter):
        G = derivative(x[0], x[1])
        x -= eta * G / np.linalg.norm(G)
        score = objective(x[0], x[1])
        solutions.append(x.copy())

        print(">%d f(%s) = %.5f" % (t, x, score))

    return solutions

In [ ]:
def SPIDER(x, objective, derivative, n_iter):
    rho = 0.95
    eta = 0.02

    solutions = list()
    solutions.append(x.copy())

    score = objective(x[0], x[1])
    M = np.array([0., 0.])
    Gprev = np.array([0., 0.])

    for t in range(n_iter):
        G = derivative(x[0], x[1])
        M = M*rho + G
        D = (G - Gprev)

        x = 0.999 * x

        x -= eta*np.sign(0.1*M + G + D)
        score = objective(x[0], x[1])
        solutions.append(x.copy())

        Gprev = G.copy()

        print(">%d f(%s) = %.5f" % (t, x, score))

    return solutions


In [ ]:
#x = bounds[:, 0] + rand(len(bounds)) * (bounds[:, 1] - bounds[:, 0])
x = np.array([0.6745, 0.9554])

# perform the gradient descent search
solutions = GradPID(x, objective, derivative, n_iter)

In [ ]:
# sample input range uniformly at 0.1 increments
xaxis = arange(bounds[0, 0], bounds[0, 1], 0.02)
yaxis = arange(bounds[1, 0], bounds[1, 1], 0.02)
# create a mesh from the axis
x, y = meshgrid(xaxis, yaxis)
# compute targets
results = np.log(objective(x, y)) + 0.01
# create a filled contour plot with 50 levels and jet color scheme

In [ ]:
fig, ax = pyplot.subplots(figsize=(19.2,10.8))
xdata, ydata = [], []
ln, = ax.plot([], [], ".-", color="w")
ax.contourf(x, y, results, levels=50, cmap="jet")
ax.set_xlim(bounds[0,:])
ax.set_ylim(bounds[1,:])

def update(frame):
    xdata.append(frame[0])
    ydata.append(frame[1])
    ln.set_data(xdata, ydata)
    return ln,

ani = FuncAnimation(fig, update, frames=solutions, blit=True)
ani.save('gradpid_bad.mp4')

In [ ]:
# PLOT SOLUTIONS

#pyplot.figure(figsize=(4,3))

#pyplot.contourf(x, y, results, levels=50, cmap="jet")
# plot the sample as black circles
#solutions = asarray(solutions)
# a_solutions = asarray(a_solutions)
#pyplot.plot(solutions[:, 0], solutions[:, 1], ".-", color="w")

# pyplot.plot(a_solutions[:, 0], a_solutions[:, 1], '.-', color='b')

#pyplot.savefig('spider_wd.svg')

# show the plot
#pyplot.show()

